In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [4]:
df=pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding='latin-1',usecols=['v1','v2'])
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.groupby('v1').describe()

v2                                                               
     count unique                                                top freq
v1                                                                       
ham   4825   4516                             Sorry, I'll call later   30
spam   747    653  Please call our customer service representativ...    4

In [6]:
df['v1'].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [7]:
df_spam=df[df['v1']=='spam']
df_spam

,v1,v2
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [8]:
df_ham=df[df['v1']=='ham']
df_ham_downsampled=df_ham.sample(df_spam.shape[0])

In [19]:
df_ham_downsampled

,v1,v2
5191,ham,"Sorry, I'll call later"
651,ham,I'm gonna say no. Sorry. I would but as normal...
2332,ham,We are both fine. Thanks
3716,ham,I'm gonna rip out my uterus.
763,ham,Nothing but we jus tot u would ask cos u ba gu...
...,...,...
5276,ham,Dunno leh cant remember mayb lor. So wat time ...
570,ham,Yar lor wait 4 my mum 2 finish sch then have l...
265,ham,Same. Wana plan a trip sometme then
4788,ham,We're on the opposite side from where we dropp...


In [9]:
df_balanced=pd.concat([df_ham_downsampled,df_spam])
df_balanced

,v1,v2
5040,ham,She ran off with a younger man. we will make p...
2425,ham,Hey!!! I almost forgot ... Happy B-day babe ! ...
4133,ham,Is it your yahoo boys that bring in the perf? ...
1636,ham,"No shit, but I wasn't that surprised, so I wen..."
1470,ham,Take some small dose tablet for fever
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [10]:
df_balanced['v1'].value_counts()

v1
ham     747
spam    747
Name: count, dtype: int64

In [24]:
df_balanced.sample(7)

,v1,v2
2988,spam,No 1 POLYPHONIC tone 4 ur mob every week! Just...
4132,spam,FreeMsg Today's the day if you are ready! I'm ...
3458,spam,Not heard from U4 a while. Call me now am here...
3928,ham,Yes. Nigh you cant aha.
1334,ham,Oh ! A half hour is much longer in Syria than ...
2900,spam,Ur cash-balance is currently 500 pounds - to m...
5102,spam,This msg is for your mobile content order It h...


In [11]:
df_balanced['spam']=df_balanced['v1'].apply(lambda x : 1 if x=='spam' else 0)
df_balanced

,v1,v2,spam
5040,ham,She ran off with a younger man. we will make p...,0
2425,ham,Hey!!! I almost forgot ... Happy B-day babe ! ...,0
4133,ham,Is it your yahoo boys that bring in the perf? ...,0
1636,ham,"No shit, but I wasn't that surprised, so I wen...",0
1470,ham,Take some small dose tablet for fever,0
...,...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...,1
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,1
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...,1
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...,1


In [22]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df_balanced['v2'],df_balanced['spam'],test_size=0.4,random_state=42)

In [64]:
x_train.head()

1597    URGENT! Your Mobile number has been awarded wi...
115     As I entered my cabin my PA said, '' Happy B'd...
4256    important information 4 orange user . today is...
5031          Ok i vl..do u know i got adsense approved..
3862    Oh my god! I've found your number again! I'm s...
Name: v2, dtype: object

In [13]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

2024-01-31 13:03:25.655503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 13:03:25.655605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 13:03:25.787709: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
get_sentences=([
    "dolalrs have i have","help the poort"
])
def  get_sentence(sentences):
    preprocessed_text=bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']


print(get_sentence(get_sentences))
    
    

tf.Tensor(
[[-0.81303227 -0.26099235 -0.17735796 ... -0.10035001 -0.6043854
   0.84591496]
 [-0.9276919  -0.4834719  -0.8577713  ... -0.39387426 -0.76728255
   0.93065304]], shape=(2, 768), dtype=float32)


In [15]:
from sklearn.metrics.pairwise import cosine_similarity



## Build a model

In [16]:
from keras.layers import Input,Dense,Dropout
from keras.models import Model

In [17]:
model_name=Input(shape=(),dtype=tf.string)

preprocessed=bert_preprocess(model_name)
output=bert_encoder(preprocessed)
output=Dropout(0.1)(output['pooled_output'])
final=Dense(1,activation="sigmoid")(output)

model=Model(inputs=[model_name],outputs=[final])

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['input_1[0][0]']             
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [19]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
history = model.fit(
    x_train, y_train,  # Training data and labels
    epochs=20,
    verbose=1,
    validation_data=(x_test, y_test)  # Validation data and labels
)


Epoch 1/20
28/28 [==============================] - 8s 289ms/step - loss: 0.1455 - accuracy: 0.9531 - val_loss: 0.1317 - val_accuracy: 0.9679
Epoch 2/20
28/28 [==============================] - 8s 286ms/step - loss: 0.1376 - accuracy: 0.9565 - val_loss: 0.1314 - val_accuracy: 0.9652
Epoch 3/20
28/28 [==============================] - 8s 284ms/step - loss: 0.1286 - accuracy: 0.9632 - val_loss: 0.1314 - val_accuracy: 0.9626
Epoch 4/20
28/28 [==============================] - 8s 285ms/step - loss: 0.1303 - accuracy: 0.9609 - val_loss: 0.1306 - val_accuracy: 0.9626
Epoch 5/20
28/28 [==============================] - 8s 283ms/step - loss: 0.1327 - accuracy: 0.9609 - val_loss: 0.1307 - val_accuracy: 0.9626
Epoch 6/20
28/28 [==============================] - 8s 284ms/step - loss: 0.1412 - accuracy: 0.9587 - val_loss: 0.1302 - val_accuracy: 0.9679
Epoch 7/20
28/28 [==============================] - 8s 283ms/step - loss: 0.1275 - accuracy: 0.9587 - val_loss: 0.1312 - val_accuracy: 0.9626
Epoch 

In [32]:
model.predict(['URGENT, IMPORTANT INFORMATION FOR O2 USER. TODAY IS YOUR LUCKY DAY! 2 FIND OUT WHY LOG ONTO HTTP://WWW.URAWINNER.COM THERE IS A FANTASTIC SURPRISE AWAITING FOR YOU'])



1/1 [==============================] - 1s 758ms/step


array([[0.9012754]], dtype=float32)

In [28]:
x_test[1406]

'URGENT, IMPORTANT INFORMATION FOR O2 USER. TODAY IS YOUR LUCKY DAY! 2 FIND OUT WHY LOG ONTO HTTP://WWW.URAWINNER.COM THERE IS A FANTASTIC SURPRISE AWAITING FOR YOU'